# 编程和测试要求
## 5. 工作模式

> 基于S-AES算法，使用密码分组链(CBC)模式对较长的明文消息进行加密。注意初始向量(16 bits) 的生成，并需要加解密双方共享。
> 
> 在CBC模式下进行加密，并尝试对密文分组进行替换或修改，然后进行解密，请对比篡改密文前后的解密结果。

In [21]:
# 加载实现的加密算法类
from SAES.SAES import SAES

### 5.1 CBC工作模式的实现

在本小节中，将选取测试用例为:

In [22]:
# CBC工作模式的测试用例
plaintext = "Hello S-AES and CBC!"
key = "1101001110100101"
iv = "0101101000001111"

#### 5.1.1 采用CBC模式的加密过程

In [23]:
def cbc_encrypt(plaintext, key, iv):
    saes = SAES(key=key)
    previous_cipherblock = iv
    ciphertext = ""

    # 转换输入为二进制
    binary_input = ''.join([bin(ord(char)).replace("0b", "").zfill(8) for char in plaintext])

    # 检查字符数并进行填充
    if len(plaintext) % 2 == 1:  # 奇数字符，添加00000001
        binary_input += '00000001'
    else:  # 偶数字符，添加00000010 00000010
        binary_input += '00000010' + '00000010'

    # 使用CBC模式进行加密
    for i in range(0, len(binary_input), 16):
        block = binary_input[i:i + 16]

        # 明文块与前一个密文块异或
        xor_block = ''.join([str(int(a) ^ int(b)) for a, b in zip(block, previous_cipherblock)])

        cipherblock = saes.encrypt(xor_block)
        ciphertext += cipherblock

        previous_cipherblock = cipherblock

    # 将结果转换为十六进制字符串
    hex_ciphertext = hex(int(ciphertext, 2))[2:].zfill(len(ciphertext) // 4)

    return hex_ciphertext


cbc_encrypt函数接收用户字符输入plaintext,约定的key以及iv初始向量。

In [24]:
ciphertext = cbc_encrypt(plaintext, key, iv)
print(f'密文是:{ciphertext}')

密文是:a95ea41bda4db7b2672fc73a5af31b90e6444fc43b63


#### 5.1.2 对CBC模式的解密过程

In [25]:
def cbc_decrypt(ciphertext, key, iv):
    saes = SAES(key=key)
    previous_cipherblock = iv
    binary_output = ""

    # 从十六进制字符串转换为二进制
    binary_input = bin(int(ciphertext, 16))[2:].zfill(len(ciphertext) * 4)

    # 使用CBC模式进行解密
    for i in range(0, len(binary_input), 16):
        block = binary_input[i:i + 16]

        decrypted_block = saes.decrypt(block)

        # 解密块与前一个密文块异或
        xor_block = ''.join([str(int(a) ^ int(b)) for a, b in zip(decrypted_block, previous_cipherblock)])
        binary_output += xor_block

        previous_cipherblock = block

    # 移除填充
    if binary_output[-8:] == '00000001':
        binary_output = binary_output[:-8]
    elif binary_output[-16:] == '00000010' + '00000010':
        binary_output = binary_output[:-16]

    # 将二进制转换回ASCII
    ascii_output = ''.join([chr(int(binary_output[i:i + 8], 2)) for i in range(0, len(binary_output), 8)])

    return ascii_output

cbc_decrypt函数接收16进制字符串,约定的key以及iv初始向量。

In [26]:
decrypted_text = cbc_decrypt(ciphertext, key, iv)
print(f'解密后的明文是:{decrypted_text}')

解密后的明文是:Hello S-AES and CBC!


测试用例通过,CBC密码分组链实现结束

#### 5.2 篡改密文组结果分析

##### 5.2.1 在加密过程中被篡改:
篡改第一个密文组末位,定义函数用以模拟此情况。

In [27]:
def cbc_encrypt_tamper(plaintext, key, iv, tamper_first_block=False):
    saes = SAES(key=key)
    previous_cipherblock = iv
    ciphertext = ""
    blocks = []  # 用于存储每个加密块，以便后续输出和比较

    # 转换输入为二进制
    binary_input = ''.join([bin(ord(char)).replace("0b", "").zfill(8) for char in plaintext])

    # 检查字符数并进行填充
    if len(plaintext) % 2 == 1:  # 奇数字符，添加00000001
        binary_input += '00000001'
    else:  # 偶数字符，添加00000010 00000010
        binary_input += '00000010' + '00000010'

    # 使用CBC模式进行加密
    for i in range(0, len(binary_input), 16):
        block = binary_input[i:i + 16]

        # 明文块与前一个密文块异或
        xor_block = ''.join([str(int(a) ^ int(b)) for a, b in zip(block, previous_cipherblock)])

        cipherblock = saes.encrypt(xor_block)

        # 篡改第一个块的最后一个bit
        if i == 0 and tamper_first_block:
            cipherblock = cipherblock[:15] + ('1' if cipherblock[15] == '0' else '0') + cipherblock[16:]
      
        ciphertext += cipherblock
        blocks.append(cipherblock)
        previous_cipherblock = cipherblock
    # 将结果转换为十六进制字符串
    hex_ciphertext = hex(int(ciphertext, 2))[2:].zfill(len(ciphertext) // 4)

    return hex_ciphertext


In [28]:
ciphertext1 = cbc_encrypt_tamper(plaintext,key,iv)
plaintext1 = cbc_decrypt(ciphertext1,key,iv)

ciphertext2 = cbc_encrypt_tamper(plaintext,key,iv,True)
plaintext2 = cbc_decrypt(ciphertext2,key,iv)

print("未被篡改的密文:",ciphertext1)
print("被篡改后的密文:",ciphertext2)
# 
print("未被篡改的解密明文:",plaintext1)
print("被篡改后的解密明文:",plaintext2)

未被篡改的密文: a95ea41bda4db7b2672fc73a5af31b90e6444fc43b63
被篡改后的密文: a95f741c6a4727b47720a7348aff8b98864f3fc11b60
未被篡改的解密明文: Hello S-AES and CBC!
被篡改后的解密明文: (kllo S-AES and CBC!


发现篡改的效果并不明显,只有在更改的block才有影响。

所以得出结论：在没有其他安全措施情况下,在加密过程中被篡改是不安全的,加密算法需要保证连续性

#### 5.2.2 加密完成后密文分组被篡改:

依然是篡改密文分组block1的末位

In [29]:
def cbc_encrypt_tamper(plaintext, key, iv, tamper_first_block=False):
    saes = SAES(key=key)
    previous_cipherblock = iv
    ciphertext = ""
    blocks = []  # 用于存储每个加密块，以便后续输出和比较

    # 转换输入为二进制
    binary_input = ''.join([bin(ord(char)).replace("0b", "").zfill(8) for char in plaintext])

    # 检查字符数并进行填充
    if len(plaintext) % 2 == 1:  # 奇数字符，添加00000001
        binary_input += '00000001'
    else:  # 偶数字符，添加00000010 00000010
        binary_input += '00000010' + '00000010'

    # 使用CBC模式进行加密
    for i in range(0, len(binary_input), 16):
        block = binary_input[i:i + 16]

        # 明文块与前一个密文块异或
        xor_block = ''.join([str(int(a) ^ int(b)) for a, b in zip(block, previous_cipherblock)])

        cipherblock = saes.encrypt(xor_block)


      
        ciphertext += cipherblock
        blocks.append(cipherblock)
        previous_cipherblock = cipherblock
    if tamper_first_block:
        first_block = blocks[0]
        tampered_block = first_block[:15] + ('1' if first_block[15] == '0' else '0')
        ciphertext = ciphertext.replace(first_block, tampered_block, 1)
    # 将结果转换为十六进制字符串
    hex_ciphertext = hex(int(ciphertext, 2))[2:].zfill(len(ciphertext) // 4)

    return hex_ciphertext


In [30]:
ciphertext1 = cbc_encrypt_tamper(plaintext,key,iv)
plaintext1 = cbc_decrypt(ciphertext1,key,iv)

ciphertext2 = cbc_encrypt_tamper(plaintext,key,iv,True)
plaintext2 = cbc_decrypt(ciphertext2,key,iv)

print("未被篡改的密文:",ciphertext1)
print("被篡改后的密文:",ciphertext2)
# 
print("未被篡改的解密明文:",plaintext1)
print("被篡改后的解密明文:",plaintext2)

未被篡改的密文: a95ea41bda4db7b2672fc73a5af31b90e6444fc43b63
被篡改后的密文: a95fa41bda4db7b2672fc73a5af31b90e6444fc43b63
未被篡改的解密明文: Hello S-AES and CBC!
被篡改后的解密明文: (klmo S-AES and CBC!


可以看到，密文在已经形成之后被篡改会再带来后一个block的更改，其余不变。

这种篡改在没有其余安全措施的情况下依然不安全，篡改攻击精度高。

### 5.3 针对CBC的攻击

#### **1. 字节反转攻击**

在CBC模式下，由于每一个密文块的解密结果会与上一个密文块进行XOR操作，我们可以利用这一特性来实施字节反转攻击。具体来说，如果我们改变某个密文块`Ciphertext-N-1`中的一个字节，那么在解密时，这会导致下一个密文块`Ciphertext-N`解密出的明文产生相应的变化，而这种变化是我们可以预测和控制的。

**举个例子**

假设我们有一个明文块`P1`，其内容为`Hello`，对应的密文块为`C1`。现在，我们想将其改变为`Jello`。

**第一步：计算差异**

首先，我们需要计算原始明文和我们想要的明文之间的差异：
$$
Δ = \text{Hello} \oplus \text{Jello}
$$

**第二步：将差异应用于C1的前一个块**

假设`C0`是`C1`的前一个密文块，我们可以通过以下方式修改`C0`：
$$
C0' = C0 \oplus Δ
$$

现在，当我们解密`C1`时，会得到：
$$
D(C1) \oplus C0' = D(C1) \oplus (C0 \oplus Δ) = (D(C1) \oplus C0) \oplus Δ
$$

由于原始的解密过程是:
$$
D(C1) \oplus C0 = P1 = \text{Hello}
$$
我们可以得到：
$$
P1' = \text{Hello} \oplus Δ = \text{Jello}
$$

**结论**

这种攻击方法之所以有效，是因为在CBC模式下，单个密文块在解密过程中只与其前一个密文块有关。这使得我们可以通过修改前一个密文块来控制当前明文块的解密结果，而不会影响其他块的解密。


### 5.4 结果分析

经过**5.2**的实例和**5.3**的分析,我们得到结论：

没有其他安全措施的CBC工作模式容易受到篡改攻击和字节替换攻击。

对于**5.2**的内容，我们进一步做形式化数学论证

# CBC模式下密文块的替换对解密过程的影响

假设有明文块序列 \({P_1, P_2, P_3, ...}\)，密文块序列 \({C_1, C_2, C_3, ...}\) 以及初始化向量 IV。

在CBC模式下，加密过程定义如下：

\begin{equation}
C_1 = E(P_1 \oplus IV)
\end{equation}
\begin{equation}
C_2 = E(P_2 \oplus C_1)
\end{equation}
\begin{equation}
C_3 = E(P_3 \oplus C_2)
\end{equation}
... (以此类推)

其中，\(E()\) 表示AES加密函数，\(\oplus\) 表示异或操作。

## 1. 在生成加密块的过程中替换

假设在生成密文块`C1`后，我们立即将其更改为`C1'`并使用它加密后续的明文块：

\begin{equation}
C_2' = E(P_2 \oplus C_1')
\end{equation}
\begin{equation}
C_3' = E(P_3 \oplus C_2')
\end{equation}
... (以此类推)

现在，我们考虑解密过程：

\begin{equation}
P_1' = D(C_1') \oplus IV
\end{equation}
对于\(P_2'\)，我们有：
\begin{equation}
P_2' = D(C_2') \oplus C_1'
\end{equation}
由于在加密`P2`时使用的是`C1'`，这意味着：
\begin{equation}
P_2' = P_2
\end{equation}

从`C3'`开始，其解密过程如下：
\begin{equation}
P_3' = D(C_3') \oplus C_2'
\end{equation}
因为`C2'`是基于`C1'`生成的，所以`P3'`可以被正确解密为`P3`。同样的逻辑适用于所有后续的块。

**结论**：只有`P1'`是错误的，所有其他的明文块都可以被正确解密。

## 2. 在密文块生成完毕之后替换

假设所有密文块都已生成，我们更改第一个密文块`C1`为`C1'`。解密过程为：

\begin{equation}
P_1' = D(C_1') \oplus IV
\end{equation}
对于\(P_2'\)，我们有：
\begin{equation}
P_2' = D(C_2) \oplus C_1'
\end{equation}

由于在加密`P2`时使用的是原始的`C1`而非`C1'`，因此`P2'`不会与原始的`P2`相等。

从`C3`开始，其解密过程为：
\begin{equation}
P_3 = D(C_3) \oplus C_2
\end{equation}
`C3`的解密不依赖于`C1'`，因此`P3`可以被正确解密。同样的逻辑适用于所有后续的块。

**结论**：`P1'`和`P2'`是错误的，但从`P3`开始，所有明文块都可以被正确解密。

